# KB API Test Notebook

This notebook tests the API call to the KB (Kungliga biblioteket) data endpoint. We'll perform the following steps:
1. Print out a single search call
2. Perform a search with all parameters and access token
3. Additional tests and error handling

In [ ]:
import requests
from urllib.parse import quote_plus
import json
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
KB_API_KEY = os.getenv('KB_API_KEY')

# Base URL for the KB API
BASE_URL = 'https://data.kb.se/search'

# Test parameters
TEST_PARAMS = {
    'to': '1908-06-30',
    'from': '1908-01-01',
    'isPartOf.@id': 'https://libris.kb.se/2ldhmx8d4mcrlq9#it',  # Svenska Dagbladet
    'q': 'konsert',
    'searchGranularity': 'part'
}

# Headers
HEADERS = {
    'Accept': 'application/json',
    'Authorization': f'Bearer {KB_API_KEY}'
}

## 1. Print out a single search call

In [ ]:
# Construct and print the URL for a single search call
query_string = '&'.join([f"{k}={quote_plus(v)}" for k, v in TEST_PARAMS.items()])
full_url = f"{BASE_URL}?{query_string}"
print(f"Single search call URL:\n{full_url}")

## 2. Perform a search with all parameters and access token

In [ ]:
def perform_search(params):
    try:
        response = requests.get(BASE_URL, params=params, headers=HEADERS)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error occurred: {e}")
        return None

# Perform the search
result = perform_search(TEST_PARAMS)

if result:
    print("Search successful!")
    print(f"Number of hits: {result.get('totalHits', 0)}")
    print("\nFirst hit details:")
    print(json.dumps(result['hits'][0], indent=2) if result['hits'] else "No hits found.")
else:
    print("Search failed. Please check your API key and parameters.")

## 3. Additional tests and error handling

In [ ]:
def test_api_robustness():
    print("Testing API robustness...\n")
    
    # Test 1: Invalid date format
    invalid_date_params = TEST_PARAMS.copy()
    invalid_date_params['to'] = '1908/06/30'
    print("Test 1: Invalid date format")
    result = perform_search(invalid_date_params)
    print("Result:", "Error as expected" if not result else "Unexpected success")
    
    # Test 2: Future date
    future_date_params = TEST_PARAMS.copy()
    future_date_params['to'] = '2050-01-01'
    print("\nTest 2: Future date")
    result = perform_search(future_date_params)
    print("Result:", "Success" if result else "Failure")
    
    # Test 3: Invalid newspaper ID
    invalid_id_params = TEST_PARAMS.copy()
    invalid_id_params['isPartOf.@id'] = 'https://libris.kb.se/invalid_id'
    print("\nTest 3: Invalid newspaper ID")
    result = perform_search(invalid_id_params)
    print("Result:", "Error as expected" if not result else "Unexpected success")
    
    # Test 4: Empty query
    empty_query_params = TEST_PARAMS.copy()
    empty_query_params['q'] = ''
    print("\nTest 4: Empty query")
    result = perform_search(empty_query_params)
    print("Result:", "Success" if result else "Failure")
    
    print("\nRobustness testing completed.")

# Run the robustness tests
test_api_robustness()

## 4. Performance Test

In [ ]:
import time

def performance_test(num_requests=5):
    print(f"Running performance test with {num_requests} requests...")
    total_time = 0
    
    for i in range(num_requests):
        start_time = time.time()
        result = perform_search(TEST_PARAMS)
        end_time = time.time()
        
        if result:
            request_time = end_time - start_time
            total_time += request_time
            print(f"Request {i+1}: {request_time:.2f} seconds")
        else:
            print(f"Request {i+1}: Failed")
    
    avg_time = total_time / num_requests
    print(f"\nAverage request time: {avg_time:.2f} seconds")

# Run the performance test
performance_test()

## Conclusion

This notebook has tested the KB API endpoint by:
1. Printing out a single search call URL
2. Performing a search with all parameters and the access token
3. Testing API robustness with various edge cases
4. Conducting a simple performance test

Review the output of each cell to ensure that the API is working as expected. If any issues are encountered, double-check your API key, parameters, and network connection.